In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import preprocess_input

In [3]:
!git clone https://github.com/laxmimerit/Movies-Poster_Dataset.git

Cloning into 'Movies-Poster_Dataset'...
remote: Enumerating objects: 7873, done.
remote: Total 7873 (delta 0), reused 0 (delta 0), pack-reused 7873
Receiving objects: 100% (7873/7873), 246.59 MiB | 38.39 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Checking out files: 100% (7871/7871), done.


In [4]:
data = pd.read_csv('./Movies-Poster_Dataset/train.csv')

In [5]:
img_width = 224
img_height = 224
X = []
for i in tqdm(range(data.shape[0])):
    path = '/content/Movies-Poster_Dataset/Images/' + data['Id'][i] + '.jpg'
    img = image.load_img(path,target_size=(img_width,img_height,3))
    img = image.img_to_array(img)
    img = img/255.0
    X.append(img)
X = np.array(X)

100%|██████████| 7254/7254 [00:27<00:00, 267.55it/s]


In [6]:
y = data.drop(['Id','Genre'],axis = 1)
y = y.to_numpy()

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0,test_size=0.15)

In [9]:
IMAGE_SIZE = [img_width,img_height]
vgg = VGG16(input_shape = IMAGE_SIZE + [3] , weights = 'imagenet' , include_top = False)


58892288/58889256 [==============================] - 0s 0us/step


In [10]:
for layer in vgg.layers:
    layer.trainable = False

In [12]:

from tensorflow.keras.layers import Input,Lambda,Flatten,Dense
x = Flatten()(vgg.output)

In [13]:
from keras.models import Model

prediction = Dense(25,activation='softmax')(x)

model = Model(inputs = vgg.input , outputs = prediction)

In [14]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [15]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [16]:
history = model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Epoch 1/5
193/193 [==============================] - 3595s 19s/step - loss: 0.3057 - accuracy: 0.3043 - val_loss: 0.3094 - val_accuracy: 0.2883
Epoch 2/5
193/193 [==============================] - 3598s 19s/step - loss: 0.2290 - accuracy: 0.4068 - val_loss: 0.3002 - val_accuracy: 0.2746
Epoch 3/5
193/193 [==============================] - 3603s 19s/step - loss: 0.1931 - accuracy: 0.4506 - val_loss: 0.3179 - val_accuracy: 0.2433
Epoch 4/5
193/193 [==============================] - 3606s 19s/step - loss: 0.1785 - accuracy: 0.4707 - val_loss: 0.3264 - val_accuracy: 0.3627
Epoch 5/5
193/193 [==============================] - 3606s 19s/step - loss: 0.1742 - accuracy: 0.4925 - val_loss: 0.3288 - val_accuracy: 0.3205
